<a href="https://colab.research.google.com/github/leegaon0622/Bigdata/blob/main/%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0_13%EC%A3%BC%EC%B0%A8_2_movie_review1119.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

과제2)
test_movie_review30.csv   파일의 label 컬럼을 채우고  

마지막 줄에는 영화 타이틀, 리뷰, label 을 채운 후  

다음  code를 수행 하여 예측 결과와 label의 차이를 확인.  
movie_review1119.ipynb와  review_results.txt 제출

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 465 kB 43.3 MB/s 


In [ ]:
from konlpy.tag import Okt

okt = Okt()
def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

In [ ]:
import pandas as pd
test_df = pd.read_csv('/content/test_movie_review30.csv', encoding='utf8')
test_df.head()

,title,review,label
0,블랙 펜서 : 와칸다 포에버,히어로 액션 영화에 히어로도 없고 액션도 없음,0
1,블랙 펜서 : 와칸다 포에버,스케일은 역시...,1
2,블랙 펜서 : 와칸다 포에버,채드윅 보스만이 그리워짐...,0
3,블랙 펜서 : 와칸다 포에버,전편만한 속편 없겠으나 주인공이 부재한 상황에서는 할수있는 최선의 선택으로 보임,1
4,블랙 펜서 : 와칸다 포에버,채드윅 보스만 빈자리가 아쉽다,0


In [ ]:
print(test_df['label'].value_counts())

1    15
0    14
Name: label, dtype: int64


한글이 아닌것은 공백으로 치환

In [ ]:
import re
test_df['review'] = test_df['review'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', "", x))

In [ ]:
print(test_df['review'].head())

0                       히어로 액션 영화에 히어로도 없고 액션도 없음
1                                         스케일은 역시
2                                   채드윅 보스만이 그리워짐
3    전편만한 속편 없겠으나 주인공이 부재한 상황에서는 할수있는 최선의 선택으로 보임
4                                채드윅 보스만 빈자리가 아쉽다
Name: review, dtype: object


토큰 생성기 load

In [ ]:
import joblib
tfidf = joblib.load('/content/tf_model.pkl')
test_tfidf = tfidf.transform(test_df['review'])

감성 분류 모델 load

In [ ]:
lr_model = joblib.load('/content/lr_model.pkl')

In [ ]:
test_predict=lr_model.predict(test_tfidf)
from sklearn.metrics import accuracy_score

print('감성 분석 정확도 : ', round(accuracy_score(test_df['label'], test_predict), 3))

감성 분석 정확도 :  0.897


In [ ]:
test_df['predicted_label']=test_predict

In [ ]:
print('### 영화별 긍정 review 수 ###')
print(test_df.groupby('title')['predicted_label'].sum())


### 영화별 긍정 review 수 ###
title
공조2                2
블랙 펜서 : 와칸다 포에버    1
비상선언               3
자백                 8
Name: predicted_label, dtype: int64


In [ ]:
mask=test_df['label']!=test_df['predicted_label']

In [ ]:
print('### 예측이 잘못된 review')
print(test_df.loc[mask,['review']])

### 예측이 잘못된 review
                                          review
3   전편만한 속편 없겠으나 주인공이 부재한 상황에서는 할수있는 최선의 선택으로 보임
8                                  내용은 그저그럼눈이 행복
18                         진행 될 수록 불편했고 마음이 복잡했다


In [ ]:
print(test_df.tail())

   title                                   review  label  predicted_label
24    자백                         반전의 반전이 계속 되는 영화      1                1
25    자백                             몰입감 높고 재미있어요      1                1
26    자백                 와 반전 나나 연기 생각보다 잘해서 놀랐네요      1                1
27    자백  원작 보고 봤는데도 쫄깃하네요 막판에 반전 부분 연출이 정말 좋았습니다      1                1
28    자백                     스페인원작의 팽팽한 긴장감이 사라졌다      0                0


In [ ]:
test_df.to_csv('202044020-이가온 review_results.txt')